In [26]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [27]:
source = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

In [28]:
sopa = BeautifulSoup(source, 'xml')

In [29]:
tabla=sopa.find('table')

In [30]:
df = pd.DataFrame(columns = ['Postalcode','Borough','Neighborhood'])

In [31]:
for cell in tabla.find_all('tr'):
    row=[]
    for cell in cell.find_all('td'):
        row.append(cell.text.strip())
    if len(row)==3:
        df.loc[len(df)] = row



In [32]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [33]:
df=df[df['Borough']!='Not assigned']

In [34]:
df[df['Neighborhood']=='Not assigned']=df['Borough']

In [36]:
df.head()

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [37]:
df2=df.groupby('Postalcode')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df2=df2.reset_index(drop=False)
df2.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)



In [39]:
df3 = pd.merge(df, df2, on='Postalcode')

In [40]:
df3.drop(['Neighborhood'],axis=1,inplace=True)

In [41]:
df3.drop_duplicates(inplace=True)

In [42]:
df3.rename(columns={'Neighborhood_joined':'Neighborhood'},inplace=True)

In [43]:
df3.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park


In [44]:
df3.shape

(103, 3)

In [57]:
dfgeo = pd.read_csv('Geospatial_Coordinates_toronto.csv')

In [58]:
dfgeo.columns = ['Postalcode','Latitude','Longitude']

In [59]:
dfgeo.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [61]:
dfnew = pd.merge(df3,dfgeo[['Postalcode','Latitude', 'Longitude']],on='Postalcode')

In [62]:
dfnew.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
